## <a name="0">Practice Classification using SVM, Logistic Regression and KNN</a>

1. <a href="#1">Problem Statement</a>
2. <a href="#2">Data Exploration</a>
3. <a href="#3">Data Processing</a>
4. <a href="#4">Feature selection</a>
5. <a href="#5">Model Training</a>
6. <a href="#6">Improvements</a>

---
## 1. <a name="1">Problem Statement</a>
(<a href="#0">Go to top</a>)
* Can you build a machine learning model to accurately predict whether or not the patients in the dataset have diabetes or not?

### Data fields

* Pregnancies : Number of times pregnant
* Glucose : Plasma glucose concentration a 2 hours in an oral glucose tolerance test
* BloodPressure : Diastolic blood pressure (mm Hg)
* SkinThickness : Triceps skin fold thickness (mm)
* Insulin : 2-Hour serum insulin (mu U/ml)
* BMI : Body mass index (weight in kg/(height in m)^2)
* DiabetesPedigreeFunction : Diabetes pedigree function
* Age : Age (years)
* Outcome : Class variable (0 or 1) 268 of 768 are 1, the others are 0

### Imports

In [ ]:
#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import RandomUnderSampler
#
from sklearn.metrics import classification_report
from sklearn.metrics import  precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import accuracy_score 
from sklearn.metrics import plot_roc_curve,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [ ]:
#---------- configs
pd.set_option('max_rows',100)
pd.set_option('max_columns',100)

In [ ]:
### loading data
df_train = pd.read_csv(r"../input/pima-indians-diabetes-database/diabetes.csv")
df_train.info()

* No missing values ! , all datatypes are numeric lets check the data

---
## 2. <a name="2">Data Exploration</a>
(<a href="#0">Go to top</a>)

In [ ]:
df_copy = df_train.copy()

---
#### Univariate Exploration

In [ ]:
sns.countplot(data = df_copy,x='Outcome')
plt.title('Diabetes Class 0 , 1 Count')
plt.show()

* outcome is not balanced

In [ ]:
numerical_columns = list(df_copy.select_dtypes(exclude=['object']).columns)

def plot_numerical_histogram(df,x_cols=[],rows = 2,cell_size = 4):
    size = len(x_cols)
    cols = size // rows
    fig,axes = plt.subplots(rows,cols,figsize=(cols * cell_size, rows * cell_size))
    fig.suptitle("Variable Histogram")
    for i,axe in enumerate(axes.flatten()):
        if(i < size):
            sns.histplot(df[x_cols[i]],ax=axe,alpha=0.4)
            median = df[x_cols[i]].median()
            axe.set_title(x_cols[i] + f' ,Median : {median:0.1f}')
            axe.axvline(median, color ='red',lw=2, alpha = 0.55)
        else:
            print('subplots > n of columns, change n of rows')
            break 
    plt.tight_layout()
    plt.show()
plot_numerical_histogram(df_copy,numerical_columns,rows=2)

* some notes 
    * 0 glucose level , 0 blood pressure ,0 BMI , 0 Insulin , 0 SkinThickness
* skin thickness : Its thickness gives information about the fat reserves of the body. ~
    * ref : https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5083983/
    * reflects the fat , maybe correlated to bmi 
    * ranges for women according to the ref , ~[10-60]
* most of the distributions are right skewed.

---
#### Bivariate Exploration

* Get Correlated Variables

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df_copy.corr(),annot=True,square=True,linewidths=1,cmap='mako',cbar=True)
plt.show()

* notable correlations:
    * Pregnancies and Age
    * Glucose and Insulin
    * Skin Thickness and Insulin , BMI
    

In [ ]:
sns.scatterplot(data=df_copy,x='Insulin',y='Glucose',hue='Outcome',palette='Set1',alpha=0.5,x_jitter=0.2,y_jitter=0.3)
plt.title('Insuline and Glucose')
plt.show()

In [ ]:
sns.scatterplot(data=df_copy,x='BMI',y='SkinThickness',hue='Outcome',palette='Set1',alpha=0.5)
plt.title('SkinThickness and BMI')
plt.show()

In [ ]:
sns.boxenplot(data=df_copy,y='Age',x='Pregnancies')
plt.title('Pregnancies and Age')
plt.show()

---
## 3. <a name="3">Data Processing</a>
(<a href="#0">Go to top</a>)

In [ ]:
# split for processing by train information
train,test = train_test_split(df_copy, test_size = 0.2,stratify=df_copy['Outcome'], random_state=47)

* we have to deal with the incorrect data -zeros-
* data skewness.
* data scaling.

In [ ]:
def encode_data_series(df_prep,df_original,col_name,from_vals,to_vals):
    df_prep[col_name] = df_original[col_name].replace(from_vals,to_vals,inplace=False)

In [ ]:
train_copy = train.copy()
test_copy = test.copy()

In [ ]:
incorrect_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI']
 
encode_data_series(train_copy,train,incorrect_cols,[0],[np.nan])
encode_data_series(test_copy,test,incorrect_cols,[0],[np.nan])

In [ ]:
train_copy.isna().sum()

* for low percentage of missing data i will fill by median

In [ ]:
low_missing = ['Glucose','BloodPressure','BMI']

for na_col in low_missing:
    encode_data_series(train_copy,train_copy,[na_col],[np.nan],train[na_col].median())
    encode_data_series(test_copy,test_copy,[na_col],[np.nan],train[na_col].median())

In [ ]:
train_copy.isna().sum()

In [ ]:
from sklearn.impute import KNNImputer
kimputer = KNNImputer(n_neighbors=10)
kimputer.fit(train_copy) 
train_imp = kimputer.transform(train_copy)
test_imp = kimputer.transform(test_copy)

In [ ]:
train_impA = pd.DataFrame(train_imp,columns = train_copy.columns)
test_impA = pd.DataFrame(test_imp,columns = test_copy.columns)

* lets visualize some of the results

In [ ]:
sns.scatterplot(data=train_impA,x='BMI',y='SkinThickness',hue='Outcome',palette='Set1',alpha=0.5)
plt.title('SkinThickness and BMI')
plt.show()

In [ ]:
sns.scatterplot(data=train_impA,x='Insulin',y='Glucose',hue='Outcome',palette='Set1',alpha=0.5,x_jitter=0.2,y_jitter=0.3)
plt.title('Insuline and Glucose')
plt.show()

In [ ]:
from scipy.stats import skew
#-- skewed data
train_impA_skewed = train_impA.copy()
test_impA_skewed = test_impA.copy()

skewed_feats = train_impA.apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

print('Skewed Features : ' ,list(skewed_feats))
train_impA_skewed[skewed_feats] = np.log1p(train_impA[skewed_feats])
test_impA_skewed[skewed_feats] = np.log1p(test_impA[skewed_feats])

In [ ]:
from sklearn.preprocessing import StandardScaler
s_scaler = StandardScaler()

traintoscale = train_impA_skewed.drop('Outcome',axis=1).copy()
testtoscale = test_impA_skewed.drop('Outcome',axis=1).copy()

s_scaler.fit(traintoscale)
train_scaled = s_scaler.transform(traintoscale)
test_scaled = s_scaler.transform(testtoscale)

train_ready = pd.DataFrame(train_scaled,columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'])
test_ready = pd.DataFrame(test_scaled,columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'])

train_ready['Outcome'] = train_impA['Outcome']
test_ready['Outcome'] = test_impA['Outcome']

In [ ]:
#some distributions after fixing skewness and standardization
plot_numerical_histogram(train_ready,skewed_feats,rows=2)

--- 
## 4. <a name="4">Feature selection</a>
(<a href="#0">Go to top</a>)

In [ ]:
df_corr = train_ready.corr()
df_sorted_corr = df_corr['Outcome'].abs().sort_values(ascending=False)[1:]
correlation_threshold =  0.2 
high_corr_cols = df_sorted_corr[df_sorted_corr > correlation_threshold].index.tolist()

In [ ]:
print('Sorted Features by Pearson Correlation','\n')
print(df_sorted_corr)

--- 
## 5. <a name="5">Model Training</a>
(<a href="#0">Go to top</a>)

In [ ]:
#-- raw data with imbalanced class

# X_train = train_ready[high_corr_cols]
# y_train = train_ready['Outcome']

#-- balancing class distribution in training data only before training
over_smote = SMOTE(sampling_strategy=0.9) # oversampling didn't bring greater results 
under_smote = RandomUnderSampler(sampling_strategy=0.6,random_state=47) #~minimizing the imbalance abit.

X_train, y_train = under_smote.fit_resample(train_ready[high_corr_cols],train_ready['Outcome'])

X_test = test_ready[high_corr_cols]
y_test = test_ready['Outcome']

In [ ]:
#original value counts
train_ready['Outcome'].value_counts()

In [ ]:
#after smote undersampling
y_train.value_counts()

---
#### KNN Classifier

In [ ]:
knn_clf = KNeighborsClassifier()

param_grid = {'n_neighbors' : [i for i in range(1,20) if i%2==0]
              ,'weights':['uniform','distance']}

knn_gcv = GridSearchCV(knn_clf,param_grid,cv=5,scoring='roc_auc')
knn_gcv_fit = knn_gcv.fit(X_train, y_train)

print('best estimator : ',knn_gcv_fit.best_estimator_)
print('best score : ',knn_gcv_fit.best_score_)

In [ ]:
y_eval = knn_gcv_fit.best_estimator_.predict(X_test)
print(classification_report(y_test, y_eval))

plot_roc_curve(knn_gcv_fit.best_estimator_,X_test,y_test)
plt.title('ROC')
plt.show()

---
#### SVC

In [ ]:
svc_clf = SVC(class_weight='balanced',random_state=42)

param_grid = {'kernel':['poly', 'rbf'],'degree':[1,2],'C' : np.arange(1,10,1)}

svc_gcv = GridSearchCV(svc_clf,param_grid,cv=5,scoring='roc_auc')
svc_gcv_fit = svc_gcv.fit(X_train, y_train)

print('best estimator : ',svc_gcv_fit.best_estimator_)
print('best score : ',svc_gcv_fit.best_score_)

In [ ]:
y_eval = svc_gcv_fit.best_estimator_.predict(X_test)
print(classification_report(y_test, y_eval))

plot_roc_curve(svc_gcv_fit.best_estimator_,X_test,y_test)
plt.title('ROC')
plt.show()

---
#### Logisitic Regression

In [ ]:
log_clf = LogisticRegression(class_weight='balanced',random_state=42)

param_grid = {'C' :  np.arange(0.01,0.3,0.01)}

log_gcv = GridSearchCV(log_clf,param_grid,cv=5,scoring='roc_auc')
log_gcv_fit = log_gcv.fit(X_train, y_train)

print('best estimator : ',log_gcv_fit.best_estimator_)
print('best score : ',log_gcv_fit.best_score_)


In [ ]:
y_eval = log_gcv_fit.best_estimator_.predict(X_test)
print(classification_report(y_test, y_eval))

plot_roc_curve(log_gcv_fit.best_estimator_,X_test,y_test)
plt.title('ROC')
plt.show()

--- 
## 6. <a name="6">Improvements</a>
(<a href="#0">Go to top</a>)

* room for improvement like : 
    * Handling outliers.
    * Feature engineering.
    * Interaction between predictors.
    * clustering for uncovering structure in the data.. and much more i may revisit in the future.